In [6]:
from torch.utils.data import Dataset
import torch
from torchvision import transforms
import pandas as pd
import numpy as np


class MNISTDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.df.iloc[idx, 1:]
        sample = {'image': np.array(image), 'label': np.array(self.df.iloc[idx, 0])}

        if self.transform:
            sample = self.transform(sample)

        return sample
    
class Rescale(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        return {'image': image/255, 'label': label}

class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        return {'image': torch.from_numpy(image).type(torch.float),
                'label': torch.from_numpy(label).type(torch.long)}

In [7]:
train_dataset = MNISTDataset(csv_file='fashion-mnist_train.csv',
                                           transform=transforms.Compose([
                                               Rescale(),
                                               ToTensor()
                                           ]))

test_dataset = MNISTDataset(csv_file='fashion-mnist_test.csv',
                                           transform=transforms.Compose([
                                               Rescale(),
                                               ToTensor()
                                           ]))

In [8]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16,
                        shuffle=True, num_workers=0)

test_dataloader = DataLoader(train_dataset, batch_size=16,
                        shuffle=True, num_workers=0)

In [18]:
import torch.nn as nn
import os
import matplotlib.pyplot as plt


def save_to_csv(model_name, epoch, train_acc, train_loss, test_acc, test_loss):
    filename = model_name+'.csv'
    data = {'epoch':[epoch], 
            'train_acc':[train_acc], 
            'train_loss':[train_loss],
            'test_acc':[test_acc],
            'test_loss':[test_loss],
    } 
    if not os.path.exists(model_name+'.csv'):
        df = pd.DataFrame(data) 
        df.to_csv(filename, index=False)
    else:
        df = pd.read_csv(filename)
        data = {k: v[0] for k, v in data.items()}
        # df = df.append(data, ignore_index = True) 
        df.loc[len(df.index)] = [epoch, train_acc, train_loss, test_acc, test_loss]  
        df.to_csv(filename, index=False)

def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100 
    return acc

def train(model, model_name, train_dataloader, test_dataloader, loss_fn, epochs = 100, lr=0.01):
    optimizer = torch.optim.SGD(params=model.parameters(), 
                            lr=lr)

    for epoch in range(epochs):
        train_acc = 0
        train_loss = 0
        test_acc = 0
        test_loss = 0
        
        for sample in train_dataloader:
            X = sample['image']
            y = sample['label']
            model.train()

            y_logits = model(X).squeeze() 
            loss = loss_fn(y_logits, y)

            train_loss += loss.item()

            y_pred = y_logits.argmax(dim=1)
            acc = accuracy_fn(y, y_pred)
            train_acc += acc

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_acc /= len(train_dataloader)
        train_loss /= len(train_dataloader)
        test_acc, test_loss = test(model, test_dataloader, loss_fn)

        save_to_csv(model_name, epoch, train_acc, train_loss, test_acc, test_loss)

        if epoch+1 % 10 == 0:
            print("Epoch:", epoch, "; Acc:", train_acc, "; Loss:", train_loss, "; test acc:", test_acc, "; test loss:", test_loss)
        
    return model

def test(model, dataloader, loss_fn):
    model.eval()
    test_acc = 0
    test_loss = 0
    with torch.inference_mode():
        for sample in dataloader:
            X = sample['image']
            y = sample['label']

            y_logits = model(X).squeeze()
            loss = loss_fn(y_logits, y)

            test_loss += loss.item()
            acc = accuracy_fn(y, y_logits.argmax(dim=1))
            test_acc += acc
            
    return test_acc/len(dataloader), test_loss/len(dataloader)

def plot_training_data(model_name, title=None, save=False):
    df = pd.read_csv(model_name+'.csv')
    fig, axs = plt.subplots(2, 2)
    if title is None:
        fig.suptitle(model_name+' results')
    else:
        fig.suptitle(title)
    axs[0, 0].set_title('Train loss')
    axs[0, 0].plot(df['epoch'].to_list(), df['train_loss'].to_list())
    axs[0, 1].set_title('Train acc')
    axs[0, 1].plot(df['epoch'].to_list(), df['train_acc'].to_list())
    axs[1, 0].set_title('Test loss')
    axs[1, 0].plot(df['epoch'].to_list(), df['test_loss'].to_list())
    axs[1, 1].set_title('Test acc')
    axs[1, 1].plot(df['epoch'].to_list(), df['test_acc'].to_list())
    fig.show()
    if save:
        fig.savefig(model_name+'_metrics'+'.png')

In [ ]:
num_epochs = 100
lr = 0.1
loss_fn = nn.CrossEntropyLoss()
results = {}

for num_layers in range(1, 4):
    for num_neurons in [8, 16, 32, 64, 128]:
        if num_layers == 0 and num_neurons != 8:
            continue
        if num_layers == 0:
            model = nn.Sequential(
                nn.Linear(784, 10),
                nn.ReLU(),
            )
            name = 'model0'
        else:
            model = nn.Sequential(
                nn.Linear(784, num_neurons),
                nn.ReLU(),
            )

            for i in range(num_layers-1):
                model.add_module(f'linear_layer{i}', nn.Linear(num_neurons, num_neurons))
                model.add_module(f'relu_layer{i}', nn.ReLU())

            model.add_module('output', nn.Linear(num_neurons, 10))
            name = f'model{num_layers}_{num_neurons}'

        model = train(model, name, train_dataloader, test_dataloader, loss_fn, epochs=num_epochs, lr=lr)
        loss, acc = test(model, test_dataloader, nn.CrossEntropyLoss())
        results[name] = {
            'test loss': loss,
            'test acc': acc
        }
        plot_training_data(name, save=True)


def stringify_results(results):
    s = ""
    for model_name, params in results.items():
        s += model_name + "\n"
        for k, v in params.items():
            if k != 'model':
                s += f'\t{k}: {v}\n'
    return s

str_results = stringify_results(results)

with open("experiment2.txt", 'w+') as f:
    f.write(str_results)
